In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split

# import pytorch for CNN model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
DEVICE

device(type='mps')

In [3]:
# import dataset from ./data/eddies_train.nc with xarray
eddies_train = xr.open_dataset('./data/eddies_train.nc')

# print the dataset
# eddies_train

# import the OSSE_U_V_SLA_SST_train.nc dataset
eddies_train = xr.open_dataset("./data/eddies_train.nc")
OSSE_test = xr.open_dataset("./data/OSSE_U_V_SLA_SST_test.nc")
OSSE_train = xr.open_dataset('./data/OSSE_U_V_SLA_SST_train.nc')
OSSE_train = OSSE_train.rename({"time_counter":"time"})



In [17]:
OSSE_train

<xarray.Dataset>
Dimensions:    (latitude: 357, longitude: 717, time: 284)
Coordinates:
    deptht     float32 ...
  * latitude   (latitude) float32 -1.084e-19 2.967 0.0 ... 3.689e+19 3.099 0.0
  * longitude  (longitude) float32 3.689e+19 -3.352 0.0 ... -3.221 0.0
  * time       (time) datetime64[ns] 2015-01-02T12:00:00 ... 2015-12-25T12:00:00
Data variables:
    vomecrtyT  (time, latitude, longitude) float32 ...
    vozocrtxT  (time, latitude, longitude) float32 ...
    sossheig   (time, latitude, longitude) float32 ...
    votemper   (time, latitude, longitude) float32 ...
Attributes:
    history:  Mon Feb 20 14:04:07 2023: ncks -A SST_2015_merged.nc SSH_U_V_20...
    NCO:      4.4.7

In [4]:
class OSSE_Dataset(Dataset):
    def __init__(self, osse = OSSE_train, eddie = eddies_train):
        
        # osse.

        self.osse = osse
        self.eddie = eddie
        
    def __len__(self):
        return len(self.osse.shape[0])
    
    def __getitem__(self, idx):


        features = self.osse[idx,:, :, :]
        label = self.eddie[idx,:, :, :]
        
        return features, label

SyntaxError: invalid syntax (3011228939.py, line 4)

In [15]:
# # create the training and testing data
# X_train, X_test, y_train, y_test = train_test_split(OSSE_train, eddies_train.eddies, test_size=0.2, random_state=42)

OSSE_train

<xarray.Dataset>
Dimensions:    (latitude: 357, longitude: 717, time: 284)
Coordinates:
    deptht     float32 ...
  * latitude   (latitude) float32 -1.084e-19 2.967 0.0 ... 3.689e+19 3.099 0.0
  * longitude  (longitude) float32 3.689e+19 -3.352 0.0 ... -3.221 0.0
  * time       (time) datetime64[ns] 2015-01-02T12:00:00 ... 2015-12-25T12:00:00
Data variables:
    vomecrtyT  (time, latitude, longitude) float32 ...
    vozocrtxT  (time, latitude, longitude) float32 ...
    sossheig   (time, latitude, longitude) float32 ...
    votemper   (time, latitude, longitude) float32 ...
Attributes:
    history:  Mon Feb 20 14:04:07 2023: ncks -A SST_2015_merged.nc SSH_U_V_20...
    NCO:      4.4.7

In [11]:
OSSE_train

<xarray.Dataset>
Dimensions:    (latitude: 357, longitude: 717, time: 284)
Coordinates:
    deptht     float32 ...
  * latitude   (latitude) float32 -1.084e-19 2.967 0.0 ... 3.689e+19 3.099 0.0
  * longitude  (longitude) float32 3.689e+19 -3.352 0.0 ... -3.221 0.0
  * time       (time) datetime64[ns] 2015-01-02T12:00:00 ... 2015-12-25T12:00:00
Data variables:
    vomecrtyT  (time, latitude, longitude) float32 ...
    vozocrtxT  (time, latitude, longitude) float32 ...
    sossheig   (time, latitude, longitude) float32 ...
    votemper   (time, latitude, longitude) float32 ...
Attributes:
    history:  Mon Feb 20 14:04:07 2023: ncks -A SST_2015_merged.nc SSH_U_V_20...
    NCO:      4.4.7

In [19]:
np.unique(eddies_train.eddies[1].values)

array([ 0.,  1.,  2., nan])

In [31]:
OSSE_train.vomecrtyT.time.shape

(284,)

In [30]:
OSSE_train.vomecrtyT.values.reshape(284, -1).shape


(284, 255969)

In [43]:
# use XGBoost to predict the eddies using the OSSE data

# import the XGBoost library
import xgboost as xgb

# import the train_test_split function from sklearn
from sklearn.model_selection import train_test_split

# import the accuracy_score function from sklearn
from sklearn.metrics import accuracy_score

# import the confusion_matrix function from sklearn
from sklearn.metrics import confusion_matrix


# create a function to train the model
def train_model(X_train, y_train, X_test, y_test):
    # create the model
    model = xgb.XGBClassifier()

    # change the nan values to 3 in y_train and y_test
    y_train = np.nan_to_num(y_train, nan=3)
    y_test = np.nan_to_num(y_test, nan=3)

    # train the model
    model.fit(X_train, y_train)
    # predict the test data
    y_pred = model.predict(X_test)
    # print the accuracy score
    print("Accuracy: %.2f%%" % (accuracy_score(y_test, y_pred) * 100.0))
    # print the confusion matrix
    print(confusion_matrix(y_test, y_pred))
    # return the model
    return model


# create a function to predict the eddies
def predict_eddies(model, X_test):
    # predict the test data
    y_pred = model.predict(X_test)
    # return the predicted eddies
    return y_pred


# create a function to plot the eddies
def plot_eddies(y_pred):
    # create the figure
    fig, ax = plt.subplots(figsize=(10, 10))
    # plot the predicted eddies
    plt.scatter(OSSE_test.longitudeT, OSSE_test.latitudeT, c=y_pred)
    # set the title
    plt.title('Predicted Eddies')
    # show the plot
    plt.show()



# create the training and testing data
X_train, X_test, y_train, y_test = train_test_split(OSSE_train.vomecrtyT, eddies_train.eddies, test_size=0.2, random_state=42)


# reshape from (time, lat, long, ...) to (284, lat*long)
y_train = y_train.values.reshape(y_train.time.values.shape[0], -1)
y_test = y_test.values.reshape(y_test.time.values.shape[0], -1)
X_train = X_train.values.reshape(X_train.time.values.shape[0], -1)
X_test = X_test.values.reshape(X_test.time.values.shape[0], -1)

# truncate the features to the first 1000
X_train = X_train[:, :1000]
X_test = X_test[:, :1000]


print(y_train.shape)

# train the model
model = train_model(X_train, y_train, X_test, y_test)

# predict the eddies
y_pred = predict_eddies(model, X_test)

# plot the eddies
plot_eddies(y_pred)


(227, 255969)


XGBoostError: [10:41:06] /Users/runner/miniforge3/conda-bld/xgboost-split_1667849614592/work/include/xgboost/objective.h:98: multioutput is not supported by current objective function
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000015142ed98 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x0000000151587274 xgboost::ObjFunction::Targets(xgboost::MetaInfo const&) const + 100
  [bt] (2) 3   libxgboost.dylib                    0x000000015154440c xgboost::LearnerConfiguration::ConfigureTargets() + 176
  [bt] (3) 4   libxgboost.dylib                    0x0000000151542560 xgboost::LearnerConfiguration::ConfigureModelParamWithoutBaseScore() + 44
  [bt] (4) 5   libxgboost.dylib                    0x0000000151537e0c xgboost::LearnerConfiguration::Configure() + 1076
  [bt] (5) 6   libxgboost.dylib                    0x000000015153806c xgboost::LearnerImpl::UpdateOneIter(int, std::__1::shared_ptr<xgboost::DMatrix>) + 144


In [7]:
print(OSSE_train.vomecrtyT.values.shape)

OSSE_train

(284, 357, 717)


<xarray.Dataset>
Dimensions:    (latitude: 357, longitude: 717, time: 284)
Coordinates:
    deptht     float32 ...
  * latitude   (latitude) float32 -1.084e-19 2.967 0.0 ... 3.689e+19 3.099 0.0
  * longitude  (longitude) float32 3.689e+19 -3.352 0.0 ... -3.221 0.0
  * time       (time) datetime64[ns] 2015-01-02T12:00:00 ... 2015-12-25T12:00:00
Data variables:
    vomecrtyT  (time, latitude, longitude) float32 nan nan nan ... 0.1552 0.2156
    vozocrtxT  (time, latitude, longitude) float32 ...
    sossheig   (time, latitude, longitude) float32 ...
    votemper   (time, latitude, longitude) float32 ...
Attributes:
    history:  Mon Feb 20 14:04:07 2023: ncks -A SST_2015_merged.nc SSH_U_V_20...
    NCO:      4.4.7

In [55]:
# CNN with 357 * 717 and 4 channels, with 4 classes in output


# create the CNN class with 3 convolutional layers
class CNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        # input 357 * 717 * 4
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(4, 16, kernel_size=3, stride=1, padding=1),
            # add batch normalization ?
            torch.nn.ReLU(),
            # torch.nn.MaxPool2d(kernel_size=2, stride=2))
            # input 357 * 717 * 16
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        # input 178 * 358 * 16
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            # output 178 * 358 * 32
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        # input 89 * 179 * 32
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 128, kernel_size=3, stride=1, padding=1),
            # output 89 * 179 * 128
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # input 44 * 89 * 128
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        # input 22 * 44 * 128
        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        # input 11 * 22 * 128
        self.layer5 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # input 5 * 11 * 128
        self.fc1 = torch.nn.Linear(5 * 11 * 128, 512, bias=True)
        self.fc2 = torch.nn.Linear(512, 128, bias=True)
        self.fc3 = torch.nn.Linear(128, 4, bias=True)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)

        return out


In [11]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 4, in_channels=4) -> None:
        super(AlexNet, self).__init__()

        self.features = nn.Sequential(
            # input 357 * 717 * 4
            nn.Conv2d(
                in_channels, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2)
            ),
            # output 89 * 179 * 64
            nn.ReLU(True),
            
            nn.MaxPool2d((3, 3), (2, 2)),
            # output 44 * 89 * 64

            nn.Conv2d(64, 192, (5, 5), (1, 1), (2, 2)),
            # output 44 * 89 * 192
            nn.ReLU(True),
            nn.MaxPool2d((3, 3), (2, 2)),
            # output 22 * 44 * 192
            nn.Conv2d(192, 384, (3, 3), (1, 1), (1, 1)),
            # output 22 * 44 * 384
            nn.ReLU(True),
            nn.Conv2d(384, 256, (3, 3), (1, 1), (1, 1)),
            # output 22 * 44 * 256
            nn.ReLU(True),
            nn.Conv2d(256, 256, (3, 3), (1, 1), (1, 1)),
            # output 22 * 44 * 256
            nn.ReLU(True),
            nn.MaxPool2d((3, 3), (2, 2)),
            # output 11 * 22 * 256
        )

        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        # output 6 * 6 * 256

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            # nn.Linear(256 * 11 * 22, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 16384),
            nn.ReLU(True),
            nn.Linear(16384, 357 * 717),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self._forward_impl(x)

    # Support torch.script function
    def _forward_impl(self, x: torch.Tensor) -> torch.Tensor:
        out = self.features(x)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.classifier(out)

        return out


In [12]:
# my_cnn = CNN().to(DEVICE)
# my_cnn(torch.randn(1, 4, 357, 717).to(DEVICE))

my_cnn = AlexNet().to(DEVICE)
my_cnn(torch.randn(1, 4, 357, 717).to(DEVICE))

my_cnn(X_train[0].values.reshape(1, 4, 357, 717).to(DEVICE))

RuntimeError: Invalid buffer size: 15.62 GB

In [68]:
pro = np.array([0, np.nan, 1, 2])

#  remove nan from pro
pro = pro[~np.isnan(pro)]
pro

array([0., 1., 2.])

In [6]:
class OSSE_Dataset(Dataset):
    def __init__(self, OSSE_tensor, eddie_tensor):
        self.OSSE_tensor = OSSE_tensor
        self.eddie_tensor = eddie_tensor

    def __len__(self):
        return self.OSSE_tensor.shape[0]

    def __getitem__(self, idx):
        features = self.OSSE_tensor[idx, :, :, :]
        label = self.eddie_tensor[idx, :, :, :]

        return features, label


In [38]:
y_train.shape

torch.Size([227, 1, 357, 717])

In [7]:
selected_var = ["vomecrtyT", "vozocrtxT", "sossheig", "votemper"]
X_full = torch.tensor(OSSE_train.get(selected_var).to_array().to_numpy())
X_full = X_full.permute(1, 0, 2, 3)
na_value = 999.0

# eddies_train = eddies_train.fillna(na_value) # fill coast with 999.
y_full = torch.tensor(eddies_train.to_array().to_numpy())
y_full = y_full.permute(1, 0, 2, 3)
nb_val = X_full.shape[0]
idx_split = int(0.8 * nb_val)

X_train = torch.tensor(X_full[:idx_split, :, :, :])
y_train = torch.tensor(y_full[:idx_split, :, :, :])
X_val = torch.tensor(X_full[idx_split:, :, :, :])
y_val = torch.tensor(y_full[idx_split:, :, :, :])
ds_train = OSSE_Dataset(X_train, y_train)

train_dataloader = DataLoader(ds_train, batch_size=16)
ds_val = OSSE_Dataset(X_val, y_val)

val_dataloader = DataLoader(ds_val, batch_size=16)


/var/folders/6r/vdmz5zjj6_d8769grj_ht__80000gn/T/ipykernel_7844/402395561.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_full[:idx_split, :, :, :])
/var/folders/6r/vdmz5zjj6_d8769grj_ht__80000gn/T/ipykernel_7844/402395561.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_full[:idx_split, :, :, :])
/var/folders/6r/vdmz5zjj6_d8769grj_ht__80000gn/T/ipykernel_7844/402395561.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val = torch.tensor(X_full[idx_split:, :, :, :])
/var/fold

In [13]:
data_iter = iter(train_dataloader)

features, labels = next(data_iter)

my_cnn = AlexNet().to(DEVICE)
my_cnn(torch.randn(1, 4, 357, 717).to(DEVICE))

my_cnn(features.to(DEVICE))

# get predicted class
pred = torch.max(my_cnn(features.to(DEVICE)), 1)[1]
pred
# nn.Softmax(dim=1)(my_cnn(features.to(DEVICE)))

RuntimeError: Invalid buffer size: 15.62 GB